In [1]:
"""
1. Wikipedia에서 검색
2. DuckDuckGo에서 검색
3. 웹사이트의 텍스트를 스크랩하고 추출합니다.
4. 리서치 결과를 .txt 파일에 저장하기
다음 쿼리로 에이전트를 실행합니다: "Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면, 에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고, DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음 .txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.
"""

from typing import Type
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import WikipediaQueryRun, BaseTool, DuckDuckGoSearchResults
from langchain.utilities import WikipediaAPIWrapper
from langchain.schema import SystemMessage
from pydantic import BaseModel, Field


class SearchToolArgsSchema(BaseModel):
    keyword: str = Field(description="The keyword you will search for.")


class CrawlToolArgsSchema(BaseModel):
    url: str = Field(description="The url you will crawl for.")


class WriteTxtToolArgsSchema(BaseModel):
    information: str = Field(description="The information wroten into txt file.")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearch"
    description = """
    Use this tool to find information about keyword in Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, keyword):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(keyword)


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearch"
    description = """
    Use this tool to find sites about keyword in DuckDuckGo.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, keyword):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(keyword)


class WebsiteCrawlTool(BaseTool):
    name = "WebsiteCrawl"
    description = """
    Use this tool to find information in sites.
    It takes a address of site as an argument.
    """
    args_schema: Type[CrawlToolArgsSchema] = CrawlToolArgsSchema

    def _run(self, url):
        return url


class WriteTxtTool(BaseTool):
    name = "WriteTxt"
    description = """
    Use this tool to write information into txt file.
    It takes arguments like "{{"filename": "xxx.txt", "information": "aaaa"}}"
    """
    arg_schema: Type[WriteTxtToolArgsSchema] = WriteTxtToolArgsSchema

    def _run(self, information):
        path = "result.txt"
        f = open(path, "w", encoding="utf-8")
        f.write(information)
        f.close()
        return path


llm = ChatOpenAI(
    temperature=1e-1,
    model_name="gpt-4o-mini",
)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebsiteCrawlTool(),
        WriteTxtTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a information collector.
            
            You find a information about keyword.

            You Wikipedia or DuckDuckGo.

            if you used DuckDuckGo, crawl informations from each websites.
            
            finally, information write into txt file named keyword.
            """
        )
    },
)

agent.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'keyword': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (pre

/Users/ryugibo/Projects/nomadcoders.gpt-challenge/.venv/lib/python3.11/site-packages/langchain/utilities/duckduckgo_search.py:82: UserWarning: backend='api' is deprecated, using backend='auto'
  results = ddgs.text(


[snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: Because the backdoor was discovered before the malicious versions of xz Utils were added to production versions of Linux, "it's not really affecting anyone in the real world," Will Dormann, a ..., title: Backdoor found in widely used Linux utility targets encrypted SSH ..., link: https://arstechnica.com/security/2024/03/backdoor-found-in-widely-used-linux-utility-breaks-encrypted-ssh-connections/], [snippet: The back door means that the connection is no longer private and could allow an attacker to insert their own text commands into the secure connection. ... This XZ backdoor was caught in Fedora Rawhide. (Although admittedly there

{'input': 'Research about the XZ backdoor',
 'output': 'I have gathered information about the XZ backdoor and saved it into a text file named "XZ_backdoor.txt". Here is a summary of the content:\n\n---\n\n# XZ Backdoor\n\n## Overview\nIn February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code on the affected Linux system. This issue has been assigned the Common Vulnerabilities and Exposures number CVE-2024-3094 and has a CVSS score of 10.0, the highest possible score.\n\n## Impact\nThe xz utility is commonly present in most Linux distributions. At the time of discovery, the backdoored version had not yet been widely deployed to production systems but was present in development versions of major distributions. The backdoor was discovered by software developer Andres

In [2]:
!cat result.txt

# XZ Backdoor

## Overview
In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code on the affected Linux system. This issue has been assigned the Common Vulnerabilities and Exposures number CVE-2024-3094 and has a CVSS score of 10.0, the highest possible score.

## Impact
The xz utility is commonly present in most Linux distributions. At the time of discovery, the backdoored version had not yet been widely deployed to production systems but was present in development versions of major distributions. The backdoor was discovered by software developer Andres Freund, who announced his findings on March 29, 2024.

## Functionality
The malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. It manipulated sshd, the executable

In [3]:
agent.invoke("Research about the XZ backdoor in DuckDuckGo")



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearch` with `{'keyword': 'XZ backdoor'}`




/Users/ryugibo/Projects/nomadcoders.gpt-challenge/.venv/lib/python3.11/site-packages/langchain/utilities/duckduckgo_search.py:82: UserWarning: backend='api' is deprecated, using backend='auto'
  results = ddgs.text(


[snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: Because the backdoor was discovered before the malicious versions of xz Utils were added to production versions of Linux, "it's not really affecting anyone in the real world," Will Dormann, a ..., title: Backdoor found in widely used Linux utility targets encrypted SSH ..., link: https://arstechnica.com/security/2024/03/backdoor-found-in-widely-used-linux-utility-breaks-encrypted-ssh-connections/], [snippet: Now that the XZ backdoor has control over sshd, attackers can possess the encryption key used to make the SSH connections hide their malicious code in the SSH login certificate and use it as an entry point to execute various mal

{'input': 'Research about the XZ backdoor in DuckDuckGo',
 'output': 'I have gathered information about the XZ backdoor and saved it in a text file named "result.txt". If you need any further details or assistance, feel free to ask!'}

In [4]:
!cat result.txt

# XZ Backdoor

## Overview
The XZ backdoor refers to malicious code that was discovered in versions 5.6.0 and 5.6.1 of xz Utils, a widely used software package for data compression. This backdoor was designed to manipulate the `sshd` executable, which is responsible for handling secure shell (SSH) connections.

## Functionality
The backdoor allows attackers to gain unauthorized remote access to systems using SSH. Once the backdoor is activated, it can intercept SSH connections, allowing attackers to possess the encryption keys used for these connections. This enables them to hide malicious code within the SSH login certificate, providing a means to execute various harmful actions on compromised devices, such as stealing files or installing additional malware like ransomware.

## Impact
Fortunately, the backdoor was discovered before the malicious versions of xz Utils were deployed in production environments, meaning it has not significantly affected users in the real world. Experts hav